# Домашняя работа Python_13

In [2]:
import pandas as pd

**Задача 1**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [8]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [11]:
rat = ratings.groupby('userId').agg(['min', 'max'])[['timestamp']].reset_index()
#rat['delta'] = rat.loc[:,'timestamp'] - rat.loc[:,'timestamp']
rat.head()

userId   timestamp            
                 min         max
0      1  1260759108  1260759205
1      2   835355395   835356246
2      3  1298861589  1298932787
3      4   949778714   949982274
4      5  1163373044  1163375145

In [26]:
rat.shape

(671, 3)

In [22]:
rat1 = ratings.groupby('userId')[['timestamp']].max()
rat2 = ratings.groupby('userId')[['timestamp']].min()

rat3 = rat2.copy(deep=True)
del rat3['timestamp']

rat3['delta'] = rat1.loc[:,'timestamp'] - rat2.loc[:,'timestamp']
rat3.head(7)

,delta
userId,
1,97
2,851
3,71198
4,203560
5,2101
6,1124022
7,2588


In [23]:
tk = ratings.groupby('userId').count() # Каждый раз пользователь ставил одну оценку одну фильму
f1 = tk.loc[:,'timestamp']
f2 = f1 > 100
print(f2)

userId
1      False
2      False
3      False
4       True
5      False
       ...  
667    False
668    False
669    False
670    False
671     True
Name: timestamp, Length: 671, dtype: bool


**Ответ к задаче 1:**

In [25]:
rat2[f2]

,timestamp
userId,
4,949778714
8,1154389340
15,997937239
17,1127468587
19,855190091
...,...
656,986240991
659,834598040
664,1343731283


In [30]:
len(rat2)

671

In [29]:
len(rat2[f2])

258

**А поможет ли np?**

In [32]:
import numpy as np

In [34]:
ratings.pivot_table(index = 'userId', values = 'timestamp', 
                    aggfunc = [np.min, np.max] ).reset_index().head()

,userId,amin,amax
,,timestamp,timestamp
0,1,1260759108,1260759205
1,2,835355395,835356246
2,3,1298861589,1298932787
3,4,949778714,949982274
4,5,1163373044,1163375145


**Есть ли функция расчёта размаха вариации???**

**Задача 2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия).

Необходимо сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [37]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [38]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [39]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [40]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [41]:
df1 = rzd.merge(auto, how = 'outer')
df1

,client_id,rzd_revenue,auto_revenue
0,111,1093.0,NaN
1,112,2810.0,NaN
2,113,10283.0,57483.0
3,114,5774.0,83.0
4,115,981.0,912.0
5,116,NaN,4834.0
6,117,NaN,98.0


In [42]:
df2 = df1.merge(air, how = 'outer')
df2

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


**Ответ к задаче 2:**

In [43]:
df3 = df2.merge(client_base, on = 'client_id', how = 'outer')
df3

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


**Задача 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

**Ответ к задаче 3:**

   Есть пользователи, таблица визитов и покупок. Покупки совершаться могли с любого устройства.
Непосредственно или удалённо. Устройство может кроме широты и долготы выдавать timestamp,
то есть фактическую дату. Привязать таблицы (соединить) таблицы можно как раз по дате совершения
покупки и в обобщённой таблице появится дополнительная информация о месте совершения покупки (широта/долгота), что позволит оценить спрос в разных точках мира на разные категории покупок.
По удалённости GPS-координат от оффлайн-магазина, категории покупок и географической зоны
можно определить необходимость присутствия на рынке конкретной категории товаров и 
смысл развивать рынок.